In [ ]:
import plotly.express as px
import requests
import datetime
import pandas as pd
import json

In [ ]:
def get_json(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        return json.loads(response.content.decode('utf-8'))
    else:
        return None

In [ ]:
record_date = datetime.date.today().strftime('%Y-%m-%d')
covid_url = 'https://covid19-api.org/api/status?date='+record_date
df_covid_worldwide = pd.io.json.json_normalize(get_json(covid_url))

print(df_covid_worldwide.tail())

In [ ]:
#Merubah format tanggal di kolom last_update jadi format tgl hari ini
df_covid_worldwide['last_update'] = pd.to_datetime(df_covid_worldwide['last_update'], format='%Y-%m-%d')
df_covid_worldwide['last_update'] = df_covid_worldwide['last_update'].apply(lambda x: x.date())
df_covid_worldwide = df_covid_worldwide.rename(columns={'country':'alpha2'})
print(df_covid_worldwide.head(10))

In [ ]:
#Mengambil data countries
countries_url = 'https://covid19-api.org/api/countries'
df_countries = pd.io.json.json_normalize(get_json(countries_url))
df_countries = df_countries.rename(columns={'alpha3': 'country_code','name':'country_name'})[['country_name','country_code','alpha2']]

print(df_countries.head())

In [ ]:
countries = []
for x in df_covid_worldwide.alpha2:
    countries.append(x)
# countries

In [ ]:
i = 0
for country in countries:
    covid_timeline_url = 'https://covid19-api.org/api/timeline/'+country
    df_world_covid_timeline = pd.io.json.json_normalize(get_json(covid_timeline_url))
    df_world_covid_timeline['last_update'] = pd.to_datetime(df_world_covid_timeline['last_update'], format='%Y-%m-%d')
    df_world_covid_timeline['last_update'] = df_world_covid_timeline['last_update'].apply(lambda x: x.date())
    if i==0:
        df_world_covid_timeline_merged = df_world_covid_timeline
    else:
        df_world_covid_timeline_merged = df_world_covid_timeline.append(df_world_covid_timeline_merged, ignore_index=True)
    i=i+1

df_world_covid_timeline_merged = df_world_covid_timeline_merged.rename(columns={'country':'alpha2'})
df_world_covid_timeline_denormalized = pd.merge(df_world_covid_timeline_merged , df_countries, on='alpha2')
df_world_covid_timeline_denormalized = df_world_covid_timeline_denormalized[(df_world_covid_timeline_denormalized['last_update'] >= datetime.date(2020, 3, 1))]
df_world_covid_timeline_denormalized.last_update = df_world_covid_timeline_denormalized.last_update.astype('str')
df_world_covid_timeline_denormalized = df_world_covid_timeline_denormalized.sort_values(by='last_update')
print(df_world_covid_timeline_denormalized.head())

In [ ]:
fig = px.choropleth(df_world_covid_timeline_denormalized,locations='country_code',color='cases',hover_name='country_name',
              animation_frame='last_update',color_continuous_scale='YlOrRd',height=700,scope='asia',
              hover_data=df_world_covid_timeline_denormalized[['cases','deaths','recovered']],
              title='Data Persebaran COVID-19 di Benua Asia Sejak 2020-03-01 s/d {}'.format(record_date))
fig.show()